<a href="https://colab.research.google.com/github/supriya130114/Salary-Project/blob/main/Smart_Lead_Scoring_Engine_(1).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

import all necessary library

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
import statsmodels.api as sm


/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


extract the dataset

In [ ]:
import pandas as pd
import io
from google.colab import files
uploaded = files.upload()


TypeError: ignored

In [2]:
data=pd.read_csv("train_wn75k28.csv")
data.head()

FileNotFoundError: ignored

In [ ]:
data.shape

data contains 39161 rows and 19 columns

lets we will check the null valuse present in our dataset if any..

In [ ]:
data.isnull().sum()

data contains some null values in two column as shown above

In [ ]:
data.info()

it gives the detail information about the data with data type,it shows that two column 'created_at' & 'signup_date' has object type of data lets we will encode it 

# data encoding

In [ ]:
from sklearn.preprocessing import LabelEncoder

In [ ]:
le=LabelEncoder()
label=le.fit_transform(data["created_at"])
label
data=data.drop("created_at",axis='columns')
data["created_at"]=label

In [ ]:
le=LabelEncoder()
label=le.fit_transform(data["signup_date"])
label
data=data.drop("signup_date",axis='columns')
data["signup_date"]=label
data

In [ ]:
data.isnull().sum()

lets we fill null values by mode function 

In [ ]:
data['products_purchased']=data['products_purchased'].fillna(data['products_purchased'].mode()[0])


In [ ]:
data.isnull().sum()

there is no any null values now in dataset

In [ ]:
data.describe()

it gives the detail description about the dataset with its std,mean and mini to max distribution of data in each column,now as some column has mini zero values in it lets we will remove it by mean of respective columns

In [ ]:
data['user_activity_var_1']=data['user_activity_var_1'].replace(0,data['user_activity_var_1'].mean())
data['user_activity_var_2']=data['user_activity_var_2'].replace(0,data['user_activity_var_2'].mean())
data['user_activity_var_3']=data['user_activity_var_3'].replace(0,data['user_activity_var_3'].mean())
data['user_activity_var_4']=data['user_activity_var_4'].replace(0,data['user_activity_var_4'].mean())
data['user_activity_var_5']=data['user_activity_var_5'].replace(0,data['user_activity_var_5'].mean())
data['user_activity_var_6']=data['user_activity_var_6'].replace(0,data['user_activity_var_6'].mean())
data['user_activity_var_7']=data['user_activity_var_7'].replace(0,data['user_activity_var_7'].mean())
data['user_activity_var_8']=data['user_activity_var_8'].replace(0,data['user_activity_var_8'].mean())
data['user_activity_var_9']=data['user_activity_var_9'].replace(0,data['user_activity_var_9'].mean())
data['user_activity_var_10']=data['user_activity_var_10'].replace(0,data['user_activity_var_10'].mean())
data['user_activity_var_11']=data['user_activity_var_11'].replace(0,data['user_activity_var_11'].mean())
data['user_activity_var_12']=data['user_activity_var_12'].replace(0,data['user_activity_var_12'].mean())
data['created_at']=data['created_at'].replace(0,data['created_at'].mean())
data['signup_date']=data['signup_date'].replace(0,data['signup_date'].mean())

In [ ]:
data.info()

In [ ]:
data.describe()

now data is well distributed

In [ ]:
data.columns

In [ ]:
sns.heatmap(data.isnull())
plt.title("Null values")
plt.show()

heat map shows that data does not contains any null values in it

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
#now lets see how data is distributed in each column
plt.figure(figsize=(20,25),facecolor='white')
plotnumber=1
for column in data:
    if plotnumber<=19:
        ax=plt.subplot(5,4,plotnumber)
        sns.distplot(data[column])
        plt.xlabel(column,fontsize=20)
    plotnumber+=1
plt.show()    

In [ ]:
# setup figure
fig, axes = plt.subplots(nrows=5, ncols=4, figsize=(16, 12))

# iterate and plot subplots
for xcol, ax in zip(data.columns[1:], [x for v in axes for x in v]):
    data.plot.scatter(x=xcol, y='buy', ax=ax, alpha=0.5, color='r')


In [ ]:
data.corr()

In [ ]:
#heat map
plt.figure(figsize=(22,7))
sns.heatmap(data.corr(),annot=True,linewidths=0.1,linecolor='black',fmt='0.2f')

it shows that correlation of each column with the target column also it gives the positive and negative correlation of data with target column,also from it is clear that some of column has zero correlation and some is havinf negative which has not that much imporatance on target column so lets we will drop it

In [ ]:
data.drop(['id','campaign_var_1','campaign_var_2','user_activity_var_6'],axis=1,inplace=True)
data.head()

lets we will check the skewness present in our dataset

In [ ]:
data.skew()

so data contains skewness lets will remove it

In [ ]:
#making the skew less than or equal to 0.55 for better prediction and plotting Normal distribution
skew=('user_activity_var_1','user_activity_var_2','user_activity_var_3','user_activity_var_4','user_activity_var_5','user_activity_var_7','user_activity_var_8','user_activity_var_9','user_activity_var_10','user_activity_var_11','user_activity_var_12')
for col in skew :
    if data.skew().loc[col]>0.55:
        data[col] = np.log1p(data[col])

In [ ]:
from scipy.stats import zscore
import numpy as np
z=np.abs(zscore(data))
threshold=3
np.where(z>3)

In [ ]:
data_new_z=data[(z<3).all(axis=1)]
data_new_z

In [ ]:
data_new_z.shape

so after removing the skewness data contains 35795 rows and 15 column in it

In [ ]:
data_new_z.skew()

In [ ]:
#plotting histogram for univariate analysis and checking the Normal Distribution
data_new_z.hist(figsize=(20,20), grid = True, layout = (5,3), bins = 30)

In [ ]:
#visualize the outliers using boxplot
plt.figure(figsize=(15,20))
graph=1
for column in data_new_z:
    if graph<15:
        ax=plt.subplot(5,3,graph)
        sns.boxplot(data_new_z[column],orient='V')
        plt.xlabel(column,fontsize=10)
        graph+=1
plt.show()        

In [ ]:
#now lets see how data is distributed in each column
plt.figure(figsize=(20,25),facecolor='white')
plotnumber=1
for column in data:
    if plotnumber<=15:
        ax=plt.subplot(5,3,plotnumber)
        sns.distplot(data[column])
        plt.xlabel(column,fontsize=20)
    plotnumber+=1
plt.show()    

now lets devide dataset into x,y variable

In [ ]:
#devide data set into feature and label
y=data['buy']
x=data.drop(['buy'],axis=1)

In [ ]:
from sklearn.preprocessing import LabelEncoder,StandardScaler

In [ ]:
#Standardize the value of x so that mean will 0 and SD will become 1 , and make the data as normal distributed
sc = StandardScaler()
sc.fit_transform(x)
x = pd.DataFrame(x,columns=x.columns)

In [ ]:
from sklearn.metrics import r2_score,mean_absolute_error,mean_squared_error
from sklearn.linear_model import LogisticRegression,Lasso,LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import AdaBoostRegressor,GradientBoostingRegressor
from sklearn.preprocessing import LabelEncoder,StandardScaler
from sklearn.model_selection import train_test_split,GridSearchCV
from sklearn.decomposition import PCA
from scipy.stats import zscore
from sklearn.model_selection import cross_val_score

In [ ]:
from sklearn.preprocessing import StandardScaler

In [ ]:
scalar=StandardScaler()
x_scaled=scalar.fit_transform(x)
x_scaled

In [ ]:
x

In [ ]:
y

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score,confusion_matrix,classification_report
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier

In [ ]:
x_train,x_test,y_train,y_test=train_test_split(x_scaled,y,test_size=0.24,random_state=40)

In [ ]:
log_reg=LogisticRegression()
log_reg.fit(x_train, y_train)
preddt=log_reg.predict(x_test)
print(accuracy_score(y_test,preddt))
print(confusion_matrix(y_test,preddt))
print(classification_report(y_test,preddt))

In [ ]:
dt=DecisionTreeClassifier()
dt.fit(x_train, y_train)
preddt=dt.predict(x_test)
print(accuracy_score(y_test,preddt))
print(confusion_matrix(y_test,preddt))
print(classification_report(y_test,preddt))

In [ ]:
rf=RandomForestClassifier()
rf.fit(x_train, y_train)
preddt=rf.predict(x_test)
print(accuracy_score(y_test,preddt))
print(confusion_matrix(y_test,preddt))
print(classification_report(y_test,preddt))

In [ ]:
kn=KNeighborsClassifier()
kn.fit(x_train, y_train)
preddt=kn.predict(x_test)
print(accuracy_score(y_test,preddt))
print(confusion_matrix(y_test,preddt))
print(classification_report(y_test,preddt))

from above various algorithm it is found that LogisticRegression() has good accuracy 97% with its suitable confusion matrix which summerises the performance of classification algorithm,it also gives the value of precision,recall,f1-score

# AUC-ROC curve

In [ ]:
from sklearn.metrics import roc_curve,roc_auc_score,plot_roc_curve

In [ ]:
disp=plot_roc_curve(log_reg,x_test,y_test)
disp=plot_roc_curve(dt,x_test,y_test,ax=disp.ax_)
disp=plot_roc_curve(rf,x_test,y_test,ax=disp.ax_)
disp=plot_roc_curve(kn,x_test,y_test,ax=disp.ax_)
plt.legend(prop={'size':11},loc='lower right')
plt.show()

from this it is found that LogisticRegression has good accuracy among all also we will cross validate the result further

In [ ]:
log_reg=LogisticRegression()

In [ ]:
log_reg.fit(x_train,y_train)

# cross validation

In [ ]:
from sklearn.model_selection import cross_val_score

In [ ]:
for i in range(2,10):
    cv=cross_val_score(log_reg,x,y,cv=i)
    print(log_reg,cv.mean())

thus we have sucessfully validated the result,we can also predict the output below

In [ ]:
#lets plot and visualize
y_pred=log_reg.predict(x_test)
y_pred

so it gives the predicted output

In [ ]:
data.tail(5)

In [ ]:
print(' buy is:',log_reg.predict(scalar.transform([[2.000000,0.423036,0.006649,0.089276,0.011272,0.423036,0.526589,0.149097,0.011311,0.000383,0.180640,0.000561,364.0,210.0]])))

# scatter plot

In [ ]:
plt.scatter(y_test,y_pred)
plt.xlabel('actual_label')
plt.ylabel('predicted_label')
plt.title('actual Vs model prediction')
plt.show()

it gives the actual VS predicted output graph

# Saving Model

In [ ]:
#saving model to the local file system
filename='finalized_model_Smart_Lead_Scoring_Engine.pickle'
pickle.dump(log_reg,open(filename,'wb'))
#prediction using the saved model
Loaded_model=pickle.load(open(filename,'rb'))
a=Loaded_model.predict(scalar.transform([[2.000000,0.423036,0.006649,0.089276,0.011272,0.423036,0.526589,0.149097,0.011311,0.000383,0.180640,0.000561,364.0,210.0]]))
a

In [ ]:
#Adjusted R2 score
log_reg.score(x_train,y_train)

In [ ]:
#check how well model fits the test data
log_reg.score(x_test,y_test)

In [ ]:
#lets plot and visualize
y_pred=log_reg.predict(x_test)
y_pred

# now lets we have test data is also provided from that we will predict the output for saved model

In [ ]:
testdata=pd.read_csv("C://Users//SAI BABA//Desktop//Job a thon//test_Wf7sxXF.csv")
testdata.head()

In [ ]:
testdata.shape

In [ ]:
testdata.info()

In [ ]:
testdata.isnull().sum()

# data encoding

In [ ]:
le=LabelEncoder()
label=le.fit_transform(testdata["created_at"])
label
testdata=testdata.drop("created_at",axis='columns')
testdata["created_at"]=label

In [ ]:
le=LabelEncoder()
label=le.fit_transform(testdata["signup_date"])
label
testdata=testdata.drop("signup_date",axis='columns')
testdata["signup_date"]=label
testdata

In [ ]:
testdata.isnull().sum()

In [ ]:
testdata['products_purchased']=testdata['products_purchased'].fillna(testdata['products_purchased'].mode()[0])

In [ ]:
testdata.isnull().sum()

In [ ]:
testdata.info()

In [ ]:
testdata.describe()

In [ ]:
testdata['user_activity_var_1']=testdata['user_activity_var_1'].replace(0,testdata['user_activity_var_1'].mean())
testdata['user_activity_var_2']=testdata['user_activity_var_2'].replace(0,testdata['user_activity_var_2'].mean())
testdata['user_activity_var_3']=testdata['user_activity_var_3'].replace(0,testdata['user_activity_var_3'].mean())
testdata['user_activity_var_4']=testdata['user_activity_var_4'].replace(0,testdata['user_activity_var_4'].mean())
testdata['user_activity_var_5']=testdata['user_activity_var_5'].replace(0,testdata['user_activity_var_5'].mean())
testdata['user_activity_var_6']=testdata['user_activity_var_6'].replace(0,testdata['user_activity_var_6'].mean())
testdata['user_activity_var_7']=testdata['user_activity_var_7'].replace(0,testdata['user_activity_var_7'].mean())
testdata['user_activity_var_8']=testdata['user_activity_var_8'].replace(0,testdata['user_activity_var_8'].mean())
testdata['user_activity_var_9']=testdata['user_activity_var_9'].replace(0,testdata['user_activity_var_9'].mean())
testdata['user_activity_var_10']=testdata['user_activity_var_10'].replace(0,testdata['user_activity_var_10'].mean())
testdata['user_activity_var_11']=testdata['user_activity_var_11'].replace(0,testdata['user_activity_var_11'].mean())
testdata['user_activity_var_12']=testdata['user_activity_var_12'].replace(0,testdata['user_activity_var_12'].mean())
testdata['created_at']=testdata['created_at'].replace(0,testdata['created_at'].mean())
testdata['signup_date']=testdata['signup_date'].replace(0,testdata['signup_date'].mean())

In [ ]:
#heat map
plt.figure(figsize=(22,7))
sns.heatmap(testdata.corr(),annot=True,linewidths=0.1,linecolor='black',fmt='0.2f')

In [ ]:
testdata.drop(['id','campaign_var_1','campaign_var_2','user_activity_var_6'],axis=1,inplace=True)
testdata.head()

In [ ]:
testdata.skew()

In [ ]:
#making the skew less than or equal to 0.55 for better prediction and plotting Normal distribution
skew=('user_activity_var_1','user_activity_var_2','user_activity_var_3','user_activity_var_4','user_activity_var_5','user_activity_var_7','user_activity_var_8','user_activity_var_9','user_activity_var_10','user_activity_var_11','user_activity_var_12')
for col in skew :
    if testdata.skew().loc[col]>0.55:
        testdata[col] = np.log1p(testdata[col])

In [ ]:
from scipy.stats import zscore
import numpy as np
z=np.abs(zscore(testdata))
threshold=3
np.where(z>3)

In [ ]:
testdata_new_z=testdata[(z<3).all(axis=1)]
testdata_new_z

In [ ]:
testdata_new_z.skew()

In [ ]:
#saving model to the local file system
filename='finalized_model_Smart_Lead_Scoring_Engine.pickle'
pickle.dump(log_reg,open(filename,'wb'))
#prediction using the saved model
Loaded_model=pickle.load(open(filename,'rb'))
a=Loaded_model.predict(testdata_new_z)
a

In [ ]:
output.to_excel("final_output.xlsx")